## Objective: Define cohort based on inclusion/exclusion criteria

#### Inclusion criteria:
- Adult patients (Age>18 years) admitted to ICU
- Outpatient prescription opioid use: 
- Hydromorphone (Vicodin)
- Oxycodone (Oxycontin, Percocet)
- Morphine 
- Fentanyl
- Tramadol
- Buprenorphine (Suboxone)
- Methadone 
- Oxymorphone (Opana)
- Meperidine (Demerol)

##### Exclusion criteria:
- Non prescription opioid use:
- Heroin 
- Fentanyl
- Death within the 1st 24 hours
- Terminal diagnosis: Cancer, anoxic brain injury
- Admission for opioid overdose 


In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from IPython.display import display, HTML # used to print out pretty pandas dataframes
display(HTML("<style>.container { width:100% !important; }</style>")) # widest display
import matplotlib.dates as dates
import matplotlib.lines as mlines

pd.options.display.max_colwidth = 500
pd.options.display.width = 500
pd.options.display.max_columns = 500
pd.options.display.max_rows = 200


%matplotlib inline
plt.style.use('ggplot') 

# specify user/password/where the database is
sqluser = 'eightiesfanjan'
sqlpass = 'squiggle'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'

query_schema = 'SET search_path to ' + schema_name + ';'

# connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)

## Inclusion Criteria
- Patients with info such as
    - Age (filtered by age >18)
    - Admitted to the ICU to the first time
    - Filter for those that did NOT die within 24 hours from first admission

### first we'll get earliest icu admits, otherwise we consider subjects in 180 day intervals

In [25]:
#get patients age

query = query_schema + """
WITH age_icu_table AS (
    SELECT ie.subject_id
        , ie.hadm_id
        , ie.icustay_id
        , admit.admittime as hospital_admit_time
        , admit.deathtime
        , ie.intime
        , ie.outtime
        , (ie.intime - admit.admittime) AS time_before_icu
        , (admit.deathtime - ie.intime) AS death_time_from_icu_admit
        , ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) AS age,
        CASE
            WHEN ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) <= 1
                THEN 'neonate'
            WHEN ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) <= 14
                THEN 'middle'
            -- all ages > 89 in the database were replaced with 300
            WHEN ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) > 100
                then '>89'
            ELSE 'adult'
            END AS ICUSTAY_AGE_GROUP,
        -- note that there is already a "hospital_expire_flag" field in the admissions table which you could use
        CASE
            WHEN admit.hospital_expire_flag = 1 then 'Y'           
        ELSE 'N'
        END AS hospital_expire_flag,
        -- note also that hospital_expire_flag is equivalent to "Is admit.deathtime not null?"
        CASE
            WHEN admit.deathtime BETWEEN ie.intime and ie.outtime
                THEN 'Y'
            -- sometimes there are typographical errors in the death date, so check before intime
            WHEN admit.deathtime <= ie.intime
                THEN 'Y'
            WHEN admit.dischtime <= ie.outtime
                AND admit.discharge_location = 'DEAD/EXPIRED'
                THEN 'Y'
            ELSE 'N'
            END AS ICUSTAY_EXPIRE_FLAG
    FROM 
        icustays ie
    INNER JOIN 
        patients pat
    ON 
        ie.subject_id = pat.subject_id
    INNER JOIN 
        admissions admit
    ON 
        ie.subject_id = admit.subject_id AND
        ie.hadm_id = admit.hadm_id
)
SELECT *
FROM age_icu_table
WHERE age > 18  AND hospital_expire_flag = 'Y'
AND death_time_from_icu_admit > '1 ::timestamp 
"""
df_demo= pd.read_sql_query(query,con)
df_demo



DatabaseError: Execution failed on sql 'SET search_path to mimiciii;
WITH age_icu_table AS (
    SELECT ie.subject_id
        , ie.hadm_id
        , ie.icustay_id
        , admit.admittime as hospital_admit_time
        , admit.deathtime
        , ie.intime
        , ie.outtime
        , (ie.intime - admit.admittime) AS time_before_icu
        , (admit.deathtime - ie.intime) AS death_time_from_icu_admit
        , ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) AS age,
        CASE
            WHEN ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) <= 1
                THEN 'neonate'
            WHEN ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) <= 14
                THEN 'middle'
            -- all ages > 89 in the database were replaced with 300
            WHEN ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) > 100
                then '>89'
            ELSE 'adult'
            END AS ICUSTAY_AGE_GROUP,
   

In [22]:
#get patients whose age between 18-99
#can only get ages less than 89. 

query = query_schema + """
SELECT * 
FROM patients
WHERE expire_flag = 1
LIMIT 50

"""
df_demo= pd.read_sql_query(query,con)
df_demo



,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,235,250,F,2164-12-27,2188-11-22,2188-11-22,NaT,1
1,241,257,F,2031-04-03,2121-07-08,2121-07-08,2121-07-08,1
2,244,261,M,2025-08-04,2102-06-29,2102-06-29,2102-06-29,1
3,246,263,M,2104-06-18,2168-06-13,2168-06-13,NaT,1
4,251,268,F,2132-02-21,2198-02-18,2198-02-18,NaT,1
5,627,664,F,2097-04-12,2179-04-03,NaT,2179-04-03,1
6,628,665,M,2052-05-20,2120-02-04,2120-02-04,NaT,1
7,629,666,F,2069-08-26,2139-11-20,NaT,2139-11-20,1
8,631,668,F,2096-08-18,2183-07-10,2183-07-10,2183-07-10,1
9,632,669,M,2121-10-20,2182-07-31,2182-07-31,2182-07-31,1


In [8]:
#get patients whose age between 18-99
#can only get ages less than 89. 

query = query_schema + """
SELECT DISTINCT(admission_location)
FROM admissions
ORDER BY 1
LIMIT 500

"""
df_demo= pd.read_sql_query(query,con)
df_demo



,admission_location
0,** INFO NOT AVAILABLE **
1,CLINIC REFERRAL/PREMATURE
2,EMERGENCY ROOM ADMIT
3,HMO REFERRAL/SICK
4,PHYS REFERRAL/NORMAL DELI
5,TRANSFER FROM HOSP/EXTRAM
6,TRANSFER FROM OTHER HEALT
7,TRANSFER FROM SKILLED NUR
8,TRSF WITHIN THIS FACILITY


In [3]:
#get patients whose age between 18-99
#can only get ages less than 89. 

query = query_schema + """

WITH first_admission_time AS
(
  SELECT
      p.subject_id,
      a.hadm_id,
      p.dob, 
      p.gender, 
      MIN (a.admittime) AS first_admittime, 
      MIN( ROUND( (cast(admittime as date) - cast(dob as date)) / 365.242,2) )
          AS first_admit_age
  FROM patients p
  INNER JOIN admissions a
  ON p.subject_id = a.subject_id
  GROUP BY p.subject_id, p.dob, p.gender,a.hadm_id
  ORDER BY p.subject_id
)
SELECT
    a.subject_id, 
    a.hadm_id,
    gender,
    dob,
    first_admit_age, 
    first_admittime,
    admittime,
    dischtime,
  (cast(dischtime as date) - cast(admittime as date)) as duration,
  CASE
      -- all ages > 89 in the database were replaced with 300
      WHEN first_admit_age > 89
          then '>89'
      WHEN first_admit_age >= 14
          THEN 'adult'
      WHEN first_admit_age <= 1
          THEN 'neonate'
      ELSE 'middle'
      END AS age_group,
  deathtime,
  admission_type, 
  diagnosis, 
  hospital_expire_flag as mortality
FROM first_admission_time a
INNER JOIN 
    admissions c
ON a.subject_id = c.subject_id


"""
df_demo= pd.read_sql_query(query,con)
df_demo



,subject_id,hadm_id,gender,dob,first_admit_age,first_admittime,admittime,dischtime,duration,age_group,deathtime,admission_type,diagnosis,mortality
0,2,163353,M,2138-07-17,0.00,2138-07-17 19:04:00,2138-07-17 19:04:00,2138-07-21 15:48:00,4,neonate,None,NEWBORN,NEWBORN,0
1,4,185777,F,2143-05-12,47.84,2191-03-16 00:28:00,2191-03-16 00:28:00,2191-03-23 18:41:00,7,adult,None,EMERGENCY,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0
2,6,107064,F,2109-06-21,65.94,2175-05-30 07:15:00,2175-05-30 07:15:00,2175-06-15 16:00:00,16,adult,None,ELECTIVE,CHRONIC RENAL FAILURE/SDA,0
3,7,118037,F,2121-05-23,0.00,2121-05-23 15:05:00,2121-05-23 15:05:00,2121-05-27 11:57:00,4,neonate,None,NEWBORN,NEWBORN,0
4,8,159514,M,2117-11-20,0.00,2117-11-20 10:22:00,2117-11-20 10:22:00,2117-11-24 14:20:00,4,neonate,None,NEWBORN,NEWBORN,0
5,9,150750,M,2108-01-26,41.79,2149-11-09 13:06:00,2149-11-09 13:06:00,2149-11-14 10:15:00,5,adult,2149-11-14 10:15:00,EMERGENCY,HEMORRHAGIC CVA,1
6,10,184167,F,2103-06-28,0.00,2103-06-28 11:36:00,2103-06-28 11:36:00,2103-07-06 12:10:00,8,neonate,None,NEWBORN,NEWBORN,0
7,11,194540,F,2128-02-22,50.15,2178-04-16 06:18:00,2178-04-16 06:18:00,2178-05-11 19:00:00,25,adult,None,EMERGENCY,BRAIN MASS,0
8,13,143045,F,2127-02-27,39.86,2167-01-08 18:43:00,2167-01-08 18:43:00,2167-01-15 15:15:00,7,adult,None,EMERGENCY,CORONARY ARTERY DISEASE,0
9,16,103251,M,2178-02-03,0.00,2178-02-03 06:35:00,2178-02-03 06:35:00,2178-02-05 10:51:00,2,neonate,None,NEWBORN,NEWBORN,0
